# MHPI hydroDL2.0 Tutorial: **dHBV1.1p**
---

This is a basic implementation of the generic differentiable modeling framework `dMG` using the HBV1.1p hydrology model
plugin from the `hydroDL2.0` repository.



Last Revision: 30 Oct. 2024

Authors; Leo Lonzarich

---

## 1. Basic Hands-off Deployment:

In this first demonstration, we show how `dMG` using a HBV1.1p physics model backbone from `hydroDL2` can be operated
in a  few steps. These are outlined as follows:

0. First, ensure that you have the correct *env* configured. To avoid manually downloading required Python packages,
create a `hydrodl` env using 

    `conda env create -f envs/hydrodl_env.yaml`.

    Once activated, confirm PyTorch installed correctly with `torch.cuda.is_available()`. If this reports false, try
    - `conda uninstall pytorch`
    - `conda install pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia`

1. Set your desired model and experiment configuration settings within a *yaml* config file.
    - For this tutorial, you can find this config located at `generic_diffModel/example/conf/dhbv_11p_config.yaml`. Note,
    this yaml is configured to reproduce dHBV1.1p benchmarks for 531 CAMELS basins, trained and tested for 9 and 10 years,
    respectively
    - For normal use, however, see `generic_diffModel/conf/<your_config_here>`.
2. Either run `python dMG/__main__.py` in your terminal, or (recommended) run the contents of `__main__.py` in the cells below.
    - This will parse your config into a dictionary, load the HBV1.1p hydrology model, and begin training or testing.


### 1.1 Create Configurations Dictionary

The first cell below will convert the configurations yaml file into a key-indexed dictionary, with keys being the
config settings. That is, if `mode: train` is set in `dhbv_11p_config.yaml`,the dictionary will yield `config['mode'] == 'train'`.
Similarly, `training: start_time: 1999/10/01` is equivalent to `config['training']['start_time'] == '1999/10/01'`.


In [ ]:
## Load in the dMG configuration file with dHBV1.1p options:
from omegaconf import DictConfig, OmegaConf

import hydra


# Example configs stored in /example/conf
CFG_PATH = '../conf'
CFG_NAME = 'dhbv_11p_config'



def load_config(relative_cfg_path: str, cfg_name: str) -> DictConfig:
    """ Initialize Hydra and parse model configuration yaml(s) into config dict. """
    with hydra.initialize(config_path=relative_cfg_path, version_base='1.3'):
        cfg = hydra.compose(config_name=cfg_name)
   
    cfg_dict = OmegaConf.to_container(cfg, resolve=True)
    return cfg_dict

config = load_config(CFG_PATH, CFG_NAME)


### 1.2 Run `__main__.py` with Configurations

This code instantiates a model Trainer which will train or test a model per the user's specification in the config. Note
that `__main__.py` is trimmed-down here to illustrate it's primary objective.
Within the Trainer itself, 
- CAMELS data will be loaded and preprocessed,
- A differenial model object with the HBV1.1p backbone will be created, and 
- An optimizer and loss function will be initialized.

These and other details/structure of `dMG` will be illustrated in the second part of this tutorial.


In [ ]:
import sys
sys.path.append('../../dMG') # Add the root directory of dMG to the path
sys.path.append('../../example')

import torch
import logging
from typing import Any, Dict
from conf.config import ModeEnum
from trainers import build_handler
from core.utils import (create_output_dirs, randomseed_config, set_system_spec,
                        show_args)

log = logging.getLogger(__name__)


def run_train_test(config_dict: Dict[str, Any]) -> None:
    """
    Run training and testing as one experiment.
    """
    # Training
    config_dict['mode'] = ModeEnum.train
    train_experiment_handler = build_handler(config_dict)
    train_experiment_handler.run()

    # Testing
    config_dict['mode'] = ModeEnum.test
    test_experiment_handler = build_handler(config_dict)            
    test_experiment_handler.dplh_model_handler = train_experiment_handler.dplh_model_handler
    test_experiment_handler.run()


def run_experiment(config_dict: Dict[str, Any]) -> None:
    """ Run an experiment based on the mode specified in the configuration. """
    experiment_handler = build_handler(config_dict)
    experiment_handler.run()



# Set device, dtype, output directories, and random seed.
randomseed_config(config['random_seed'])

config['device'], config['dtype'] = set_system_spec(config['gpu_id'])
config = create_output_dirs(config)

log.info(f"RUNNING MODE: {config['mode']}")
show_args(config)

# Run training and testing together, or one at a time.
if config['mode'] == ModeEnum.train_test:
    run_train_test(config)

else:
    run_experiment(config)

torch.cuda.empty_cache()


In [ ]:
sys.path

---

## 2. Breakdown of Intermediate Steps


### 2.1 Create Configurations Dictionary

Once again, we begin by creating a configurations dictionary.

In [ ]:
## Load in the dMG configuration file with dHBV1.1p options:
from omegaconf import DictConfig, OmegaConf

import hydra


# Example configs stored in /example/conf
CFG_PATH = '../conf'
CFG_NAME = 'dhbv_11p_config'



def load_config(relative_cfg_path: str, cfg_name: str) -> DictConfig:
    """ Initialize Hydra and parse model configuration yaml(s) into config dict. """
    with hydra.initialize(config_path=relative_cfg_path, version_base='1.3'):
        cfg = hydra.compose(config_name=cfg_name)
   
    cfg_dict = OmegaConf.to_container(cfg, resolve=True)
    return cfg_dict

config = load_config(CFG_PATH, CFG_NAME)
